In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.ensemble import RandomForestClassifier
import time
import xgboost as xgb
import lightgbm as lgb
import warnings
import pickle

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use(style='seaborn')
%matplotlib inline

In [ ]:
%%time
train_id = pd.read_csv('../input/ieee-fraud-detection/train_identity.csv')
train_tr = pd.read_csv('../input/ieee-fraud-detection/train_transaction.csv')
test_id = pd.read_csv('../input/ieee-fraud-detection/test_identity.csv')
test_tr = pd.read_csv('../input/ieee-fraud-detection/test_transaction.csv')

In [ ]:
print(train_id.info())
print(train_tr.info())

# Merging the data

In [ ]:
train_id.head()

In [ ]:
train_tr.head()

In [ ]:
train = pd.merge(train_tr, train_id, how='left', 
                 on='TransactionID', left_index=True, 
                right_index=True)

In [ ]:
test = pd.merge(test_tr, test_id, how='left',
               on='TransactionID', left_index=True,
               right_index=True)

In [ ]:
del train_tr, train_id, test_tr, test_id

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
target = train['isFraud']
target

In [ ]:
def differentcolumns(traincols, testcols):
    for i in traincols:
        if i not in testcols:
            print(i)
differentcolumns(train.columns, test.columns)

In [ ]:
test = test.rename(columns={"id-01": "id_01", "id-02": "id_02", "id-03": "id_03", 
                            "id-06": "id_06", "id-05": "id_05", "id-04": "id_04", 
                            "id-07": "id_07", "id-08": "id_08", "id-09": "id_09", 
                            "id-10": "id_10", "id-11": "id_11", "id-12": "id_12", 
                            "id-15": "id_15", "id-14": "id_14", "id-13": "id_13", 
                            "id-16": "id_16", "id-17": "id_17", "id-18": "id_18", 
                            "id-21": "id_21", "id-20": "id_20", "id-19": "id_19", 
                            "id-22": "id_22", "id-23": "id_23", "id-24": "id_24", 
                            "id-27": "id_27", "id-26": "id_26", "id-25": "id_25", 
                            "id-28": "id_28", "id-29": "id_29", "id-30": "id_30", 
                            "id-31": "id_31", "id-32": "id_32", "id-33": "id_33", 
                            "id-34": "id_34", "id-35": "id_35", "id-36": "id_36", 
                            "id-37": "id_37", "id-38": "id_38"})
test.head()

In [ ]:
differentcolumns(train.columns, test.columns)

# Missing Values

In [ ]:
train.isnull().sum().sum()

In [ ]:
train.isnull().sum()

In [ ]:
train.isnull().count()

In [ ]:
train.isnull().sum().sort_values(ascending=False)

In [ ]:
def getnulls(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum() / data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['total', 'precent'])
    return missing_data

In [ ]:
# Train Data Missing Values
missing_data_train = getnulls(train)
missing_data_train.head(350).T

In [ ]:
# Test Data Missing Values
missing_data_test = getnulls(test)
missing_data_test.head(350).T

In [ ]:
# Get the columns that we have to drop
droppedcols = missing_data_train[missing_data_train['total'] > 100000].index
droppedcols

In [ ]:
# Drop the columns
train.drop(droppedcols, axis=1, inplace=True)
test.drop(droppedcols, axis=1, inplace=True)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
missing_data_train = getnulls(train)
missing_data_train.head(200).T

In [ ]:
missing_data_test = getnulls(test)
missing_data_test.head(200).T

In [ ]:
train['P_emaildomain']

In [ ]:
train['P_emaildomain'].value_counts()

# See if there is some pattern in this column or just it's useless and get rid of it

In [ ]:
target.value_counts()

In [ ]:
cntgmail = 0
cntyahoo = 0
cnthotmail = 0
cntanon = 0
cntaol = 0
for i in range(len(train)):
    if train['P_emaildomain'][i] == 'gmail.com' and train['isFraud'][i] == 1:
        cntgmail += 1
    elif train['P_emaildomain'][i] == 'yahoo.com' and train['isFraud'][i] == 1:
        cntyahoo += 1
    elif train['P_emaildomain'][i] == 'hotmail.com' and train['isFraud'][i] == 1:
        cnthotmail += 1
    elif train['P_emaildomain'][i] == 'anonymous.com' and train['isFraud'][i] == 1:
        cntanon += 1
    elif train['P_emaildomain'][i] == 'aol.com' and train['isFraud'][i] == 1:
        cntaol += 1
    
print("GMAIL:", cntgmail)
print("YAHOO:", cntyahoo)
print("HOTMAIL:", cnthotmail)
print("ANON:", cntanon)
print("AOL:", cntaol)

# There is no specific pattern in this columns if there is some specific domain is always fraud or something.
# So it's time to get rid of it i guess.

In [ ]:
train.drop(['P_emaildomain'], axis=1, inplace=True)
test.drop(['P_emaildomain'], axis=1, inplace=True)

In [ ]:
print(train.shape)
print(test.shape)

# We can not figure out some pattern in this floating number columns cuz it has no meaning for us, so let's get rid of some of them and replace the nulls at others.

In [ ]:
droppedcols = missing_data_train[missing_data_train['total'] > 15000].index

In [ ]:
train.drop(droppedcols, axis=1, inplace=True)
test.drop(droppedcols, axis=1, inplace=True)
print(train.shape)
print(test.shape)

In [ ]:
differentcolumns(train.columns, test.columns)

# Concatinating all data to fill the null values with the mean and mode

In [ ]:
ntrain = len(train)
ntest = len(test)
all_data = pd.concat([train, test], axis=0, sort=False)
all_data.shape

In [ ]:
ntrain + ntest

In [ ]:
all_data.head()
all_data.to_csv(index=False)

In [ ]:
all_data.info()

In [ ]:
all_data_cols = all_data.columns
for i in all_data_cols:
    if all_data[i].dtype == 'object':
        all_data[i] = all_data[i].fillna(all_data[i].mode()[0])

In [ ]:
missing_data = getnulls(all_data)
missing_data.head(110).T

In [ ]:
all_data.head()

In [ ]:
for i in all_data_cols:
    if (i.startswith("C") or (i.startswith("V"))) and all_data[i].isnull().sum() > 0:
        all_data[i] = all_data[i].fillna(all_data[i].mode())
missing_data = getnulls(all_data)
missing_data.head(10).T

In [ ]:
all_data['card3'].value_counts()

In [ ]:
all_data['D1'].value_counts()

In [ ]:
all_data['D1'] = all_data['D1'].fillna(all_data['D1'].mode())
all_data['card3'] = all_data['card3'].fillna(all_data['card3'].mean())


In [ ]:
all_data['card2'].value_counts()

In [ ]:
all_data['card2'] = all_data['card2'].fillna(all_data['card2'].mean())

In [ ]:
missing_data = getnulls(all_data)
missing_data.head(5).T

In [ ]:
all_data['card5'].value_counts()

In [ ]:
all_data['card5'] = all_data['card5'].fillna(all_data['card5'].mean())

In [ ]:
missing_data = getnulls(all_data)
missing_data.head(5).T

# All the null values have been handled

In [ ]:
print(ntrain)
print(ntest)
print(ntrain+ntest)
print(all_data.shape)

In [ ]:
del all_data

In [ ]:
# Split the data again
train = all_data[:ntrain]
test = all_data[ntrain:]

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
test.head()

In [ ]:
print(test['isFraud'].value_counts())
test.drop(['isFraud'], axis=1, inplace=True)

In [ ]:
print(test.shape)

# Correlations

In [ ]:
corr = train.corr()
plt.subplots(figsize=(20, 20))
sns.heatmap(corr, annot=True)

# Okay this is just terrifing, we gotta figure some way to visualize it.

In [ ]:
train.head()

In [ ]:
templst = ['isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 
           'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'D1', 
           'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C6', 'C8', 'C9', 'C10', 
           'C11', 'C12', 'C13', 'C14']
corr1 = train[templst].corr()
plt.subplots(figsize=(15, 15))
sns.heatmap(corr1, annot=True)

# Hmmmmm It seems like there is no features have a strong correlation with the target feature, so we gotta keep as much feature as we can to take over that
# We are not going to delete more features

# Concatinating the data again to encode the categorical features

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat([train, test], axis=0, sort=False)
print(all_data.shape)

In [ ]:
del train,test

In [ ]:
ntrain + ntest

In [ ]:
# Encoding the data
all_data = pd.get_dummies(all_data)
print(all_data.shape)
all_data.head()

In [ ]:
# Split the data again and get ready for training
train = all_data[:ntrain]
test = all_data[ntrain:]
train.drop(['TransactionID'], axis=1, inplace=True)
test_id = test['TransactionID']
test.drop(['TransactionID'], axis=1, inplace=True)
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
target.value_counts()

In [ ]:
differentcolumns(train.columns, test.columns)

In [ ]:
train.drop(['isFraud'], axis=1, inplace=True)
test.drop(['isFraud'], axis=1, inplace=True)
print(train.shape)
print(test.shape)

# Modelling

In [ ]:
# XGBOOST
xgmodel = xgb.XGBClassifier(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=50, 
                             min_child_weight=1.7817, n_estimators=200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, random_state =7, nthread = -1,
                           tree_method='gpu_hist')
xgmodel.fit(train, target)

In [ ]:
y_pred_xg = xgmodel.predict_proba(test)

In [ ]:
y_pred_xg

In [ ]:
y_pred_xg.shape

In [ ]:
y_pred_xg[:, 1]

In [ ]:
sub = pd.DataFrame()
sub['TransactionID'] = test_id
sub['isFraud'] = y_pred_xg[:, 1]
sub.to_csv('submission_xgb.csv', index=False)